In [8]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime


In [9]:
url="https://mars.nasa.gov/news"
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup.prettify())

In [10]:
# results are returned as an iterable list
results = soup.find('ul', class_="item_list")
print(results)

None


In [11]:
mars_dict={}

# Initialize browser
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    #executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    executable_path = {'executable_path': 'chromedriver.exe'}
    return Browser("chrome", **executable_path, headless=False)

In [12]:
def scrape_start(url1):
    # Initialize browser
    browser = init_browser()
    # Visit Nasa.gov news website  
    browser.visit(url1)
    # Scrape page into soup
    html = browser.html
    global soup
    soup = BeautifulSoup(html, "html.parser")
    
def scrape_news(url):
    scrape_start(url1=url)
    soup1=soup
    res = soup1.find("section", class_="grid_gallery module list_view")
    #print (res.prettify())
    #Identify and return first news title
    news_title = res.find('div', class_="content_title").a.text
    #Identify and return first news paragraph
    news_p = res.find('div', class_="article_teaser_body").text
    print(news_title)
    print (news_p)
    
scrape_news(url="https://mars.nasa.gov/news")

Mars New Home 'a Large Sandbox'
With InSight safely on the surface of Mars, the mission team is busy learning more about the spacecraft's landing site.


In [17]:
def scrape_img(url):
    #Scrape main page
    scrape_start(url1=url) #Call scraping start function
    soup1=soup
    res1 = soup1.find('a', class_="button fancybox") #get url behind 'Full Image' button on main page
    #create link to detailed page with large image
    url2="https://www.jpl.nasa.gov" + res1["data-link"]

    #Scrape page with large (full size) image
    scrape_start(url1=url2) #Call scraping start function
    soup_img=soup
    res_img=soup_img.find('figure', class_="lede").find('a').get('href') #get url for large sized image
    featured_image_url="https://www.jpl.nasa.gov" + res_img
    print (featured_image_url)
    return featured_image_url

scrape_img(url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18848_hires.jpg


'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18848_hires.jpg'

In [34]:
def scrape_tweet(url):    
        scrape_start(url1=url)
        soup1=soup
        # Find section
        res = soup.find("a", class_="js-tweet-text-container")
        
        #Identify and return first Mars weather tweet
        mars_weather = res.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
        print (mars_weather)
        #Add to dictionary
        mars_dict["mars_weather"] = mars_weather
        return mars_dict

AttributeError: 'NoneType' object has no attribute 'find'

In [27]:
url="http://space-facts.com/mars/"
tables = pd.read_html(url)
df=pd.DataFrame({'fact':tables[0][0],'value':tables[0][1]})
table_html=df.to_html()
table_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>fact</th>\n      <th>value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td

In [30]:
def scrape_hem(url):
    #Scrape main page
    scrape_start(url1=url) #Call scraping start function
    soup1=soup
    res1 = soup1.find_all('div', class_="description") #get url behind 'Full Image' button on main page
    hemisphere_image_urls=[]
    for r in res1:
        d = {}
        name=r.a.h3.text.replace(" Enhanced","")
        link=r.a.get('href')
        #create link to detailed page with large image
        url2="https://astrogeology.usgs.gov" + link
        #Scrape page with large (full size) image
        scrape_start(url1=url2) #Call scraping start function
        soup_img=soup
        res_img=soup_img.find('div', class_="downloads").find('ul').find('li').find('a').get('href')
        img_url=res_img
        d['title'] = name
        d['img_url']=img_url
        hemisphere_image_urls.append(d)
    print (hemisphere_image_urls)
    mars_dict["hemisphere_image_urls"] = hemisphere_image_urls
    return mars_dict

scrape_hem(url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")

[{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


{'hemisphere_image_urls': [{'title': 'Cerberus Hemisphere',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
  {'title': 'Schiaparelli Hemisphere',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
  {'title': 'Syrtis Major Hemisphere',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
  {'title': 'Valles Marineris Hemisphere',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]}

In [33]:
mars_dict["hemisphere_image_urls"][0]["img_url"] 

'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'

In [35]:
    def scrape_hem(url):
        #Scrape main page
        scrape_start(url1=url) #Call scraping start function
        soup1=soup
        res1 = soup1.find_all('div', class_="description") #get url behind 'Full Image' button on main page
        hemisphere_image_urls=[]
        for r in res1:
            d = {}
            name=r.a.h3.text.replace(" Enhanced","")
            link=r.a.get('href')
            #create link to detailed page with large image
            url2="https://astrogeology.usgs.gov" + link
            #Scrape page with large (full size) image
            scrape_start(url1=url2) #Call scraping start function
            soup_img=soup
            res_img=soup_img.find('div', class_="downloads").find('ul').find('li').find('a').get('href')
            img_url=res_img
            d['title'] = name
            d['img_url']=img_url
            hemisphere_image_urls.append(d)
        print (hemisphere_image_urls)
        mars_dict["hemisphere_image_urls"] = hemisphere_image_urls
        return mars_dict

    scrape_hem(url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")

[{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


{'hemisphere_image_urls': [{'title': 'Cerberus Hemisphere',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
  {'title': 'Schiaparelli Hemisphere',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
  {'title': 'Syrtis Major Hemisphere',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
  {'title': 'Valles Marineris Hemisphere',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]}

In [ ]:
!jupyter nbconvert --to script config_template.ipynb